**CSI 4106 Introduction to Artificial Intelligence** <br/>
*Assignment 1: Data Preparation*

# Identification

Name: Rina Osman<br/>
Student Number: 300222206

# Exploratory Analysis

## Import important libraries

In [2]:
import pandas as pd
import numpy as np
import sys
# print(sys.executable)

In [3]:
# Load the dataset
url = "https://raw.githubusercontent.com/rinaxosman/csi4106-a1/main/dermatology_database_1.csv"
dataset = pd.read_csv(url)


## Guidelines

The following are the questions for Assignment 1. Under each question, we have provided an initial code cell. You are encouraged to add additional code cells to maintain logical separation of your code. For instance, place the definition of a function in one cell and its execution in a subsequent cell. This approach will help preserve clarity and enhance readability by avoiding the inclusion of excessive code within a single cell.

1. **Analysis of Missing Values**: Examine the datasets to identify and assess ssing values in various attributes. Missing values may be represented by symbols ch as '?', empty strings, or other placeholders.

    <b>1.1 In the list of options, what are the datasets that contain missing values? ecifically, which attribute or attributes has missing values? </b>

    - The dermatology dataset (dataset #2) contains missing values represented by the symbol "?". Specifically, the "Age" attribute has missing values. There is a total of 8 missing instances for the age attribute.

    <b> 1.2 Describe the methodology used for this investigation, and provide the rresponding code.</b>

   - Since we already know that the missing values are filled with "?" and only in the Age column, we applied the following code to find out how many values are missing:

In [4]:
# Counts missing val in age column
missing_age = dataset['age'].value_counts().get('?', 0)
print("Missing values in Age column:", missing_age)

Missing values in Age column: 8


<b>1.1 Data imputation involves replacing missing or incomplete data with substituted values to preserve the dataset's integrity for subsequent analysis. Propose imputation strategies for each attribute with missing values.</b>

Since a portion of the "Age" attribute is missing, we can impute them by replacing the missing values ("?") with the average age of all the available ages in the dataset. Using the mean is helpful because it gives a central value that represents the ages of most patients. Here’s the code to fill in the missing values:

In [5]:
# Impute missing values with mean age
mean_age = dataset[dataset['age'] != '?']['age'].astype(float).mean()
dataset['age'] = dataset['age'].replace('?', mean_age)

2. **Select and familiarize yourself with a classification task:** Choose one of e provided datasets for further investigation. It is advisable to select a dataset ntaining a sufficiently large number of examples, ideally around 1,000, to ensure bust results when applying machine learning algorithms in the subsequent assignment.

    <b>2.1 What is the objective of the task? Is it intended for a specific plication? Do you possess expertise in this particular domain of application?</b>

    - The goal of this task is to figure out which type of erythemato-squamous disease a patient has using the Dermatology Dataset. These diseases include psoriasis, seborrheic dermatitis, lichen planus, pityriasis rosea, chronic dermatitis, and pityriasis rubra pilaris. They all have similar signs, like redness and scaling of the skin, which makes it tough to tell them apart without tests.
    - The dataset includes clinical information about the patients and results from looking at skin samples under a microscope. It's by analyzing this data that they hope to make diagnosing these diseases easier and more accurate.
    - I don’t have a background in dermatology, but I find this topic interesting. Working with this dataset will help me learn more about how ML can be used in healthcare.s


3. **Attribute Analysis**: 

    3.1 Determine which attributes lack informativeness and should be excluded to prove the effectiveness of the machine learning analysis. If all features are emed relevant, explicitly state this conclusion.

    3.2 Examine the distribution of each attribute (column) within the dataset. Utilize histograms or boxplots to visualize the distributions, identifying any underlying patterns or outliers.

In [6]:
# Your code goes here

4. **Class Distribution Analysis**: Investigate the distribution of class labels within the dataset. Employ bar plots to visualize the frequency of instances for each class, and assess whether the dataset is balanced or imbalanced.

In [7]:
# Your code goes here

5. **Preprocessing**: 

    5.1 For numerical features, determine the best transformation to use. Indicate e transformation that seems appropriate and why. Include the code illustrating how  apply the transformation. For at least one attribute, show the distribution before d after the transformation. See [Preprocessing data](https://scikit-learn.org/able/modules/preprocessing.html).

    5.2 For categorical features, show how to apply [one-hot encoding](https://ikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).  your dataset does not have categorical data, show how to apply the one-hot encoder  the label (target variable).

In [8]:
# Your code goes here

6. **Training and target data**: Set the Python variable `X` to designate the data and `y` to designate the target class. Make sure to select only the informative features.

In [9]:
# Your code goes here

7. **Training and test sets**: Split the dataset into training and testing sets. Reserve 20% of data for testing.

In [10]:
# Your code goes here

--------------------------------------------------------------------------

# References

Make sure you provide references to ALL sources used (articles, code, algorithms).

## AI transcript
**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".